# 0. Structure du NoteBook :

## 0.1. Intro :
Ce NoteBook Python est dédié au développement d'une stratégie de "trading algorithmique". Il alternera cellule de texte expliquant l'utilité/le fonctionnement des cellules de code. Son but final sera d'être une stratégie rentable, et donc déployable avec de l'argent réel.

## 0.2. Structure :
Ce notebook sera divisé en plusieurs parties :

1. Le commencement de tout programme : Importation de librairies, acquisition des données etc.
2. Le calcul d'une série d'indicateurs boursiers, nous permettant d'obtenir des informations grâce à l'analyse technique.
3. La création de l'algorithme de régression
4. La création de données tests
5. Les backtests de la stratégie, qui quant à eux nous permettrons finalement de tester les performances de celle-ci.

## 0.3 : Termes utilisés :

Note : L'ensemble des termes boursiers utilisés / indicateurs calculés peuvent être retrouvées sur ce site : https://www.investopedia.com/ .

* Tendance haussière / bullish trend : Lorsque les prix sont en augmentation sur une certaine période.
* Tendance baissière / bearish trend : Lorsque les prix sont en baisse sur une certaine période.
* Point d'entrée / entry point, aussi appelé ouverture de position : Fait d'acheter une action.
* Point de sortie / exit point, aussi appelé fermeture de position : Fait de vendre une action.
* Avoir une position sur une action : Fait de posséder cette action (et donc d'avoir la possibilité de la vendre).
* Volatilité : Ampleur des variations de cours que subis une action. Au plus elle est élevée, au plus les possibilité de rendement/perte sont grandes. Permet de quantifier le risque d'une action à un moment donné. Attention, on dit d'un indicateur financier qu'il est "réactif" lorsqu'il est sensible aux variations de cours.
* Marché suracheté/survendu (overbought/oversold) : Lorsque beaucoup d'investisseurs achètent/vendent une action, on dit que cette action est surachetée/survendue et son prix à tendance à augmenter/diminuer.
* EURONEXT : Principal marché boursier en Europe
* Données OHLCVT : Ensemble de certaines données d'un cours sur un intervalle donné. O = Open, prix à l'ouverture de l'intervalle ; H = High, prix le plus haut sur l'intervalle ; L = Low, prix le plus faible sur l'intervalle ; C = Close, prix à la fermeture de l'intervalle ; V = Volume, volume échangé ; T = Turnover.

# 1. Getting started :

## 1.1. Importing libraries :
On commence par importer toutes les librairies que nous utiliserons durant le programme.

*   pandas et pandas_datareader pour pouvoir créer des DataFrame qui nous permettront de calculer de manières très efficaces tous nos indicateurs.
*   datetime qui nous permettra de stipuler les dates de début et de fin de période, nous permettant de choisir les données utilisées.
*   numpy pour certains calculs mathématiques.
*   time qui nous permettra de timer nos fonctions, afin de déterminer leur effcacités.
*   matplotlib et matplotlib.pyplot pour effectuer certains plot de graphiques, même si ceux effectués à travers pandas sont plus que bons.
*   On set up les valeurs par défaut de certains paramètres des graphiques. (taille et grille)

In [ ]:
# Importing libraries
%matplotlib inline 
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (12, 9)
mpl.rcParams['axes.grid'] = True

## 1.3. Mining Data :
On sait que pour notre programme, nous avons besoins de données financières/boursières. Pour les obtenir, nous utiliserons les données fournies par le site "Quandl" (src = https://www.quandl.com/ ) car les données des entreprises appartenant à l'EURONEXT sont gratuites sur ce site, et je suis fort susceptible de n'utiliser que ces données.

---

Note : Pour utiliser les dernières données disponible sur le site de Quandl, il faut posséder une clé de leur API, qui s'obtient en créant un compte gratuit. Ceux qui possèdent un compte peuvent indiquer leur clé dans la cellule ci-dessous pour utiliser les dernières données disponible. Pour ceux qui n'en possède pas, un fichier échantillon sera utilisé. Ce fichier reprend les données OHLCVT de l'action INGA depuis le 2 janvier 2019 jusqu'au 8 juillet 2020 en prenant les valeurs jour pas jour.

In [ ]:
# Data Mining

# Tickers
symbols = ["EURONEXT/INGA", "EURONEXT/PROX", "EURONEXT/SOLB"]

# Interval of time needed :
from_date = dt.date(2019,1,1)
to_date = dt.date(2020,1,1)

# Quandl API key if provided or data sample if not :
quandl_key = None
sample_url = {"EURONEXT/INGA" : "https://raw.githubusercontent.com/MonkD3/L-ambition/master/Sample_data/EURONEXT-INGA.csv",
              "EURONEXT/PROX" : "https://raw.githubusercontent.com/MonkD3/L-ambition/master/Sample_data/EURONEXT-PROX.csv",
              "EURONEXT/SOLB" : "https://raw.githubusercontent.com/MonkD3/L-ambition/master/Sample_data/EURONEXT-SOLB.csv"
              }

# Creating a data_mining() function, returning a dict with DataFrames
def data_mining(symbols, from_date, to_date = dt.datetime.now(), quandl_key = None):
  dic = {}
  for symbol in symbols :
    if quandl_key is not None :
      dic[symbol] = np.round(pd.DataFrame(pdr.quandl.QuandlReader(symbol, from_date, to_date, api_key=quandl_key).read()).iloc[::-1],2)
    else :
      global sample_url
      dic[symbol] = np.round(pd.read_csv(sample_url[symbol], index_col="Date", parse_dates=True)[from_date:to_date:-1],2)
  return dic

dic = data_mining(symbols, from_date, to_date)

# Choose wich ticker you want to analyse in the next cells
ticker = "EURONEXT/INGA"
df = dic[ticker]
close = df["Last"]
close.plot(label = "Close")
plt.legend()

## 1.4. Defining functions that determine where lines crosses each other :

Nous allons finalement programmer une fonction capable de déterminer quand est ce qu'une courbe passe au dessus d'une autre : crossover(), et une autre capable de déterminer quand est ce qu'une courbe passe en dessous d'une autre : crossunder().
Ceci nous permettra généralement de pouvoir déterminer les points d'entrées et points de sortie d'un marché.

In [ ]:
# crossover() function :

def crossover_points(curve1, curve2):
  """ Determine where curve1 crosses over curve2.
      curve1 and curve2 are DataFrame objects or Series objects
  """
  differ = curve1 - curve2
  signe = differ.apply(np.sign)
  croisement = signe.diff()
  crossover_index = croisement.index[croisement > 0].tolist()
  return crossover_index

def crossover(curve1, curve2):

  val1 = float(curve1[-1])
  val2 = float(curve2[-1])

  if val1 > val2 :
    return True
  
  else :
    return False

def crossunder(curve1, curve2):
  val1 = float(curve1[-1])
  val2 = float(curve2[-1])

  if val1 < val2 :
    return True
  
  else :
    return False

# crossunder() function :
def crossunder_points(curve1, curve2):
  """ Determine where curve1 crosses under curve2.
      curve1 and curve2 are DataFrame objects or Series objects
  """
  differ = curve1 - curve2
  signe = differ.apply(np.sign)
  croisement = signe.diff()
  crossunder_index = croisement.index[croisement < 0].tolist()
  return crossunder_index

# 2. Starting computation of indicators :
Pour plus de clarté, nous utiliserons le terme "intervalle" pour UNE donnée (un intervalle pouvant être un jour, une heure ou même une seconde suivant la vitesse à laquelle on effectue le trading), et le terme "période" pour déterminer plusieurs intervalles, typiquement la durée entre 2 jours appartenant à des mois différents. En effet le terme intervalle sera utilisé pour déterminer le temps entre deux prises de mesures, alors que le terme période sera utilisé pour déterminer un nombre d'intervalles.
## 2.1. SMA indicator :
### 2.1.1. What is it ? :
L'indicateur SMA (simple moving average, moyenne arithmétique mobile si traduit littéralement) permet de lisser les données. En supposant que les données reçues suivent une distribution normale (gaussienne) cet indicateur nous permet d'obtenir une moyenne de la valeur des n dernières intervalles ne réagissant pas à la volatilité sur la période. Ceci deviendra plus clair lors de la lecture de l'indicateur "Bande de Bollinger".
### 2.1.2. Computation of the SMA :
Le calcul est assez simple, pour calculer la moyenne mobile simple sur n intervalles et à la position i, il suffit de calculer la moyenne arithmétique des i-n-1 dernières données.
$$SMA_i = \frac{\sum_{i-n-1}^{i}Prix_i}{n} $$

### 2.1.3 : What it's used for ? :
Cet indicateur est utile pour déterminer les variations de valeur à court terme. En effet, nous pouvons déduire logiquement que les moyennes mobiles simples (SMA) sur des périodes longues (nombreux intervalles), seront beaucoup moins réactives que les SMA sur des périodes courtes. De ce fait, lorsque les SMA "courtes" passent au dessus des SMAS "longues", nous pourrons en déduire que la valeur de l'action augmentera à court terme. Habituellement les périodes utilisées pour cet indicateurs sont de 20 intervalles pour les "courtes" et 50 intervalles pour les "longues".

In [ ]:
# SMA indicators :

# Standard Time period SMAs
smas = [3,5,7,9,11,12,13,15,20,25,26,30,35,40,45,50,60,70,80,90,100]

def SMA_indicator(df, smas):
  close = df["Last"]
  for sma in smas :
    # Error handling, in case ther's missing data : print an error message
    try :
      # Giving a column name in the DataFrame
      column_name = "Sma_{}".format(sma)
      #
      # Initiating sma Computation
      #
      df[column_name] = round(close.rolling(sma).mean(), 2)
      #
      # Finishing sma conputation
      #
    except :
      print("An error has occured during the computation of SMA_{}".format(sma))
      
# Executing with default values :
SMA_indicator(df, smas)

# Plotting Sma_50, Sma_20 and close value :
df["Sma_50"].plot(label = "Sma_50")
df["Sma_20"].plot(label = "Sma_20")
close.plot(label = "Close")

# Plotting entry and exit points :
entry_points = crossover_points(df["Sma_20"], df["Sma_50"])
exit_points = crossunder_points(df["Sma_20"], df["Sma_50"])
plt.scatter(entry_points, df["Sma_20"][entry_points], label = "Buy-signal")
plt.scatter(exit_points, df["Sma_20"][exit_points], label = "Sell-signal")

plt.legend()

# Note : You can plot whatever sma you want, feel free to play with it.

## 2.2. EMA indicator :
### 2.2.1. What is it ? :
L'indicateur EMA (exponential moving average) est assez similaire aux SMA, à l'exception que ceux-ci sont beaucoup plus réactifs aux dernières données. En effet dans les SMA, nous donnons exactement le même poids à chaque valeur d'une période. Le calcul d'une EMA nous permet en fait de donner un poids plus élevé aux dernières valeurs dans le temps (C'est une sorte de somme pondérée).
### 2.2.2. Computation of the EMA :
Avant de commencer le calcul, il faut déterminer le poids k que nous allons utiliser, celui-ci ne dépend que du nombre d'intervalles choisis N et peut donc être calculé avant le reste.
$$k = \frac{2}{N + 1}$$
Ensuite, l'EMA à l'indice i peut être calculé récursivement comme :
$$ EMA_i = Prix_i * k + EMA_{i-1}*(1-k)$$
### 2.2.3. What it's used fort :
Les EMA sont utilisés exactement de la même manières que les SMA, sauf que comme ceux-ci "laggent" moins (ils sont plus réactifs aux variations des derniers intervalles) ils sont plus susceptibles de prédire la variation de tendance à court terme que les SMA. 
Un exemple d'utilisation pourrait être d'anticiper une variation de tendance auux moyen d'EMA, et de confirmer ou infirmer celle_ci avec les SMA. Vu qu'ils sont plus réactifs, ils donneront plus souvent de faux-positifs, pouvant résulter en des pertes. Il faut également faire attention à leur utilisation en cas de période de forte volatilité (pour les mêmes raisons que ci-dessus).

In [ ]:
# EMA indicators :

# Standard Time period EMAs
emas = [3,5,7,9,11,12,13,15,20,25,26,30,35,40,45,50,60,70,80,90,100]

def EMA_indicator(df, emas):
  close = df["Last"]
  for ema in emas :
    # Error handling
    try :
      # Ginving a column name for the DataFrame
      column_name = "Ema_{}".format(ema)
      #
      # Initiating ema Computation
      #
      df[column_name] = round(close.ewm(span = ema, adjust = False).mean(),2)
      #
      # Finishing ema conputation
      #
    except :
      print("An error has occured during the computation of EMA_{}".format(ema))

# Executing with default values : 
EMA_indicator(df, emas)

# Plotting Ema_20 and Ema_7 :
df["Ema_20"].plot(label = "Ema_20")
df["Ema_7"].plot(label = "Ema_7")
close.plot(label = "Close")

# Plotting entry and exit points :
entry_points = crossover_points(df["Ema_7"], df["Ema_20"])
exit_points = crossunder_points(df["Ema_7"], df["Ema_20"])
plt.scatter(entry_points, df["Ema_7"][entry_points], label = "Buy-signal")
plt.scatter(exit_points, df["Ema_7"][exit_points], label = "Sell-signal")

plt.legend()
plt.legend()

## 2.3. MACD indicator :
### 2.3.1. What is it ? :
L'indicateur MACD (Moving Average Convergence Divergence) est une combinaisons de certaines EMA. Il contient 3 courbes :
* Une "zero-line" ou "baseline" qui comme son nom l'indique est une droite d'équation $y = 0$
* Une "MACD-line" qui sera une combinaison de 2 EMA de période différentes (mais plus longue que la période de la signal-line).
* Une "signal-line" qui sera composée de l'EMA de la MACD-line sur une courte période. Cet indicateur permet de déterminer des points d'entrée et des points de sortie du marché.

Note : La MACD-line est parfois représentée comme un histogramme, représentant plus clairement les points d'entrées et les points de sortie.

### 2.3.2. Computation of the MACD :
La première étape est de choisir ses 3 périodes d'EMA. Les périodes standard utilisées sont 9 pour la signal line, 12 et 26 pour la MACD-line.
Le calcul de l'indicateur est ensuite plutôt simple :
$$MACD = EMA^{12}(Prix) - EMA^{26}(Prix) \\ Signal = EMA^{9}(MACD)$$

Il suffit en fait de construire la MACD-line sur deux EMA en soustrayant l'EMA de longue période à celle de période moyenne, et de prendre ensuite l'EMA de la MACD line sur la période souhaitée.
### 2.3.3. What it's used for ? :
Comme écrit précédemment, son utilité réside dans le fait de pouvoir déterminer des périodes d'entrée et de sortie. Lorsque la MACD-line passe au dessus de la signal-line, on appelle ça un signal d'entrée et lorsqu'elle passe en dessous, un signal de sortie. Il permet également de déterminer la "force" d'une tendance. Au plus la MACD-line est éloignée de la baseline, au plus la tendance (croissante ou décroissante) sera susceptible d'être forte. Une dernière utilité est également de déterminer si un marché est suracheté (les prix seront susceptibles de croître rapidement) ou survendu (les prix seront susceptibles de décroître rapidement). Ceci peut être observé par la pente que prend la MACD-line (la dérivée en fait, mais comme ce sont des données discrètes, il est malvenu de parler de dérivée). Si la pente est fort positive, le marché sera suracheté et si elle est fort négative, le marché sera survendu.

In [ ]:
# MACD Indicators :

def MACD_indicator(df, time_periods = [9, 12, 26]):
  time_periods.sort()

  # Error handling :
  if len(time_periods) > 3 :
    new_time_periods = time_period[:2]
    time_periods = copy(new_time_periods)
    del new_time_periods
    print("""You entered too many EMA's, the program only used : {periods}""".format(periods = time_periods))
  elif len(time_periods) < 3 :
    time_periods = [9,12,26]
    print("""You didn't entered enough EMA's, the program used standard periods : {period} """.format(periods = time_periods))

  error = False
  emas_to_compute = []
  for ema in time_periods :
    try :
      test = df["Ema_{}".format(ema)]
      del test
    except KeyError :
      error = True
      emas_to_compute.append(ema)
 
  if error :
    EMA_indicator(emas_to_compute, df)

  signal,m,l = time_periods
  # On sait que la signal line est déjà calculée, c'est une EMA
  mid = df["Ema_{}".format(m)]
  lon = df["Ema_{}".format(l)]

  # MACD si considérée en courbe
  df["MACD"] = mid - lon
  df["Signal-line"] = round(df["MACD"].ewm(span = signal, adjust = False).mean(),2)

# Executing with standard values
time_periods = [9,12,26]
MACD_indicator(df, time_periods)

# Plotting lines :
df["MACD"].plot(label = "MACD")
df["Signal-line"].plot(label = "Signal-line")
close.plot(label = "Close")
plt.axhline(y=0)

# Plotting entry and exit points :
entry_points = crossunder_points(df["Signal-line"], df["MACD"])
exit_points = crossover_points(df["Signal-line"], df["MACD"])
plt.scatter(entry_points, df["MACD"][entry_points], label = "Buy-signal")
plt.scatter(exit_points, df["MACD"][exit_points], label = "Sell-signal")
plt.legend()

## 2.4. Standard Deviation indicator :

### 2.4.1. What is it ? :
Comme son nom l'indique, l'indicateur STD est en fait l'écart-type des données d'une période de n intervalles. Si l'on fait toujours l'hypothèse que les données suivent une distribution normale (comme au point 2.1.1.), cet indicateur nous permet de déterminer l'écart type "mobile" d'une période de n intervalles en calculant l'écart type de $n - \alpha + 1$ sous-périodes contenant $\alpha$ intervalles

### 2.4.2. Computation of the STD indicator :
Le calcul de l'indicateur STD avec $\alpha$ intervalle se fait par itérations. On calcule tout d'abord l'écart-type des données de la premières sous-période, sois les $\alpha$ premières valeurs (allant de $Prix_0$ à $Prix_{\alpha}$) disons $\sigma_1$. Ensuite on se décale d'une donnée (on va maintenant de  $Prix_1$ à $Prix_{\alpha + 1}$) et on calcule l'écart-type de cette sous-période $\sigma_2$ et l'on recommence jusqu'à épuisement des données. On se retrouve alors avec $n-\alpha$ écart-types, en effet on peut facilement déduire que pour les $\alpha-1$ premières données, il sera impossible d'obtenir un écart type comprenant $\alpha$ données. 

La formule mathématique ressemble à ceci :
$$\sigma_i = \sqrt{\frac{\sum_{j = i-\alpha}^{i} (Prix_i - SMA^{\alpha}_i)^2}{\alpha}}$$
### 2.4.3. What it's used for ? :
Cet indicateur sert principalement à déterminer la volatilité du marché par rapport aux dernières valeurs. Ceci permet en fait de déterminer une plage de prix sur laquelle la marché est susceptible de se diriger. Par exemple, si l'on suit une distribution normale et que nous nous trouvons au temps i, il y'a 68% de chances que le prochain prix soit comprit dans une plage d'un écart-type. Soit $P(Prix_i - \sigma_i \leq Prix_{i+1} \leq Prix_i + \sigma_i) = 0.68$ où P(...) représente la probabilité qu'un événement se produise. Il y'a également 95% de chances que ce prix soit compris dans une plage de deux écarts-types, et 99.7% de chances qu'il soit compris dans une plage de 3 écarts types.

In [ ]:
# Standard deviation indicator (Mainly used in Bollinger's Band):

def STD_indicator(df, window = 20):
  close = df["Last"]
  df["STD_{}".format(window)] = close.rolling(window).std()

# Plotting :
STD_indicator(df)
df["STD_20"].plot(label = "STD_20")
close.plot(label = "Close")
plt.legend()

## 2.5. Bollinger's Bands indicator :

### 2.5.1. What is it ? :
Les bandes de Bollinger sont en fait une utilisation plus représentative de l'écart-type calculé ci-dessus. Elles consistent à représenter visuellement les plages de prix possibles. Elle sont généralement utilisées avec une distance de 2 écarts-types ( pour avoir une probabilité de 95% ).

### 2.5.2. Computation of Bollinger's Bands :
La première étape est de déterminer le nombre d'intervalle que l'on souhaite utiliser n. Une fois que l'on a déterminé cet intervalle, on peut calculer la moyenne mobile simple (SMA) sur la période avec ce nombre d'intervalles. On calcule ensuite l'écart-type (STD) avec le même intervalle et on obtient les bandes de Bollinger comme suit :
* $UpperBand_i = SMA^n_i + 2*STD^n_i$
* $MiddleBand_i = SMA^n_i$
* $LowerBand_i = SMA^n_i - 2*STD^n_i$

### 2.5.3. What it's used fort ? :
Les bandes de Bollinger ont exactement la même utilité que l'indicateur STD. 

In [ ]:
# Bollinger band indicator :

def Bollinger_band(df, window = 20, mu = 2):
  try :
    ma = df["Sma_{}".format(window)]
  except KeyError:
    SMA_indicator([window], df)
  finally :
    ma = df["Sma_{}".format(window)]

  STD_indicator(df, window)
  st_dev = df["STD_{}".format(window)]
  df["Boll_up"] = ma + mu*st_dev
  df["Boll_down"] = ma - mu*st_dev

# Plotting
Bollinger_band(df)
df["Boll_up"].plot(label = "Boll_up")
df["Boll_down"].plot(label = "Boll_down")
df["Sma_20"].plot(label = "Sma_20")
close.plot(label = "Close")
plt.legend()

## 2.6 : Stochastic indicator :

### 2.6.1. What is it ? :
Le stochastique est un indicateur comparant le prix actuel avec les prix antérieurs, il permet de déterminer un retournement de tendance. Cet indicateur est composé de deux courbes, que l'on appelera %k, le stochastique rapide et %D, le stochastique lent. L'indicateur stochastique est un indicateur extrêmement réactif, il faut donc faire attention à la manière de l'utiliser, car il pourrait donner des faux-positif.

Le stochastique rapide est en fait dérivé des plus hauts prix atteints sur un intervalle et des plus bas également atteint. Celui-ci n'intégrant aucunes moyenne, il est nécéssaire de le "filtrer" à l'aide d'une seconde courbe, le stochastique lent, qui est la moyenne mobile simple du stochastique rapide (généralement sur 3 intervalles).
### 2.6.2. Computation of the Stochastic :
Le stochastique rapide se calcule comme suit :
$$ \%k_i = 100 * \frac{Prix_i - PrixMin^n_i}{PrixMax^n_i - PrixMin^n_i} $$ Où $PrixMin^n_i$ représente le prix minimum des n intervalles précédant l'intervalle i, et respectivement $PrixMax^n_i$ représente le maximum. Cet indicateur se calcule en pourcentage.

Le stochastique lent est quant à lui la SMA sur n intervalle du stochastique rapide.
### 2.6.3. What it's used for ? :
Le stochastique est utilisé pour déterminer un changement de tendance à court terme. Lorsque le stochastique se situe au dessus de 80%, on dira que le marché est suracheté et donc dominé par une tendance haussière. Respectivement quand il se situe en dessous de 20%, le marché est dit survendu et la tendence sera baissière. Le stochastique peut rester plusieurs jours dans une zone extrême, confirmant une certaine inertie dans les cours. 

Signaux donnés :
* D'achat si : 
1. le stochastique rapide passe au dessus du stochastique lent tout en sortant de la zone des 20%.
2. Le stochastique rapide passe au dessus de la barre des 50%.

* De vente si :
1. Le stocahstique rapide passe en dessous du stochastique lent tout en sortant de la zone des 80%. 
2. Le stochastique rapide passe en dessous de la barre des 50%.

Il faut néanmoins toujours faire attention aux faux-positifs, qui interviendront lors de l'utilisation avec des petites périodes.

In [ ]:
# Stochastic indicators :


def Stochastic_indicator(df, window = 14):
  # Taking high and low values
  high = df["High"] ; low = df["Low"]
  close = df["Last"]

  # Taking highs and lows in a 14 days window
  High = high.rolling(window).max()
  Low = low.rolling(window).min()

  # Computing the indicator
  df["Stoch_k"]= 100 * ( (close - Low)/(High - Low) )
  df["Stoch_D"]= df["Stoch_k"].rolling(window).mean()

# Plotting
Stochastic_indicator(df)
df["Stoch_k"].plot(label = "Stochastique rapide")
df["Stoch_D"].plot(label = "Stochastique lent")
close.plot(label = "Close")

plt.legend()

## 2.7. Relative Strength Index indicator (RSI) :

### 2.7.1. What is it ? :
Le RSI est un indicateur permettant de déterminer la "Force" d'une tendance, de la même manière que le stochastique présenté précédemment. La différence entre le RSI et le stochastique est la réactivité. Le stochastique n'intégrant pas de moyenne celui-ci est très réactif aux variations des cours, le RSI quant à lui intègre une EMA, ce qui lui permet de rester réactif aux nouveaux changements tout en gardant une certaine résistance à la volatilité.

### 2.7.2. Computation of the RSI :
Avant tout il faut calculer le vecteur des variations sur la période étudiée. De ce vecteur nous allons déterminer le vecteur des hausses (up) qui aura la même taille que le vecteur des variations mais ne comptera que les valeurs positives de ce dernier et le vecteur des baisses (down), qui aura la même taille que le vecteur des variations mais qui ne compteras que les valeurs absolues des entrées négatives de ce dernier.
$$ Var_i = Prix_i - Prix_{i-1} \\ Up_i = Var_i \text{ if } Var_i > 0 \text{ else } = 0 \\ Down_i = |Var_i| \text{ if } Var_i < 0 \text{ else } = 0$$
Exemple : $$Var = [-1,0.5,0.2,-0.6] \\ Up = [0, 0.5, 0.2,0] \\ Down = [1, 0, 0, 0.6]$$

La seconde étape est de calculer les EMA des vecteurs Up et Down, que l'on nommera ici AvgUp et AvgDown. Une fois cela fait il ne reste plus qu'à appliquer ce calcul :
$$ RSI = 100*\frac{AvgUp}{AvgUp + AvgDown} $$

### 2.7.3. What it's used for ? :
Le RSI est utilisé comme le stochastique, il sert à déterminer un changement de tendance. Lorsque le RSI est aux alentours de 70 et plus, le marché est sujet à une correction baissière. Et lorsqu'il se trouve aux alentours de 30 et moins, le marché sera sujet à une correction haussière. Il peut être très intéressant de prendre en compte la convergence/divergence d'indicateurs RSI sur plusieurs périodes (comme le MACD) 





In [ ]:
# RSI indicators

def RSI_indicator(df, windows = [5,7,9,11,13,15]):
  # Computing the net change in regard of the precedent value
  close = df["Last"]
  change = close.diff()

  # Isolating Upward moves and Downward moves
  Up = np.maximum(change, np.zeros(len(change)))
  Down = abs(np.minimum(change, np.zeros(len(change))))
  # Creating an intermediary Dataframe for Computation
  RSI = pd.DataFrame({"Up": Up, "Down":Down})

  # Starting Computation of the RSI
  for window in windows :
    column_name = "RSI_{}".format(window)
    AvgUp = RSI["Up"].ewm(alpha = 1/window, adjust = False).mean()
    AvgDown = RSI["Down"].ewm(alpha = 1/window, adjust = False).mean()
    RSI[column_name] = 100*( AvgUp / (AvgUp + AvgDown))

  # Taking the mean of all the windown computed 
  # and deleting the intermediary DataFrame
  df["RSI"] = RSI.iloc[5:,2:].mean(axis = 1)
  del RSI

# Plotting
RSI_indicator(df)
df["RSI"].plot(legend = "RSI")
close.plot(legend = "Close")
plt.legend()

# 3. Design of the regression algorithm :
Maintenant que nous avons calculé une suite d'indicateurs et construit le training set, nous pouvons enfin écrire l'algorithme qui tentera de trouver des dépendances entre les variables. Ce que nous allons faire est un algorithme de régression linéaire, qui suis le même principe qu'une régression linéaire en statistique, la différence étant que nous n'allons pas nous limiter à une seule dimension. Le but de l'algorithme sera de *prédire* une valeur prochaine, sur base des valeurs précédentes. Mathématiquement, cela revient à minimiser une fonction, que nous appelerons fonction de coût et qui sera définie comme suit :
$$ L(y, \hat{y}) = \frac{1}{2}(y - \hat{y})^2 $$ 
Où $y$ est la valeur effective (celle qui s'est produite) et $\hat{y}$ est la valeur prédite par l'algorithme. $\hat{y}$ sera quant à lui défini comme une somme pondérée des variables données, dans notre cas ces variables représenteront les prix historiques et les valeurs des indicateurs. $$ \hat{y} = w_0*x_0 + w_1*x_1 + ... + w_n*x_n + b$$
que l'on peut traduire vectoriellement par :
$$ \hat{y} = \mathbf{w^Tx} + b $$
Lorsque l'on place toute nos prédictions l'une à la suite de l'autre de cette manière :
$$ \hat{y_0} = w_0*x_0^0 + w_1*x_1^0 + ... + w_n*x_n^0 + b \\ \hat{y_1} = w_0*x_0^1 + w_1*x_1^1 + ... + w_n*x_n^1 + b \\ ...$$
Nous pouvons transformer le système linéaire en une équation matricielle :
$$ \mathbf{\hat{y}} = \mathbf{Xw} + b $$
Avec $\mathbf{\hat{y}}$ le vecteur de nos prédictions.

Nous allons définir tout cela grâce à Tensorflow.


In [ ]:
days_forecasting = 2 # Number of days we want to predict.
memory_length = 10 # Number of days we want to forecast price

values = df["Last"] # Taking close values as example

features = np.array([[values[i+j] for i in range(len(values)-days_forecasting - memory_length, len(values)-days_forecasting)] for j in range(days_forecasting)])
labels = np.array(values[-days_forecasting:])

In [ ]:
# Construct a way to read data. code snippet taken from the book d2a (Dive into DeepLearning)
def load_array(data_arrays, batch_size, is_train=True):
  """Construct a TensorFlow data iterator."""
  dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
  if is_train:
      dataset = dataset.shuffle(buffer_size=1000)
  dataset = dataset.batch(batch_size)
  return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# Initialisation of the coeffictions following a gaussian distribution.
coeffs = tf.initializers.RandomNormal(stddev=0.01)

# Defining the model, this model will contain only one layer, as described before.
# We do not really need to build a model for a case this simple, but it will be very useful when things get more complex.
neural_net = tf.keras.Sequential()
neural_net.add(tf.keras.layers.Dense(1, kernel_initializer = coeffs)) # "Dense" is a fully connected layer, as described by the matrix-vector operation above

# Loss Function
loss = tf.keras.losses.MeanSquaredError()

# Training algorithm, this algorithm uses Stochastic Gradient Descent.
trainer = tf.keras.optimizers.SGD(learning_rate=0.0001)

# 4. Construction of the training set :
Nous savons que pour entraîner le modèle de l'algorithme de régression que nous allons créer ensuite, il nous faut des "exemples". Les exemples sont en fait une suite de données réelles, accompagnée de la valeur que l'algorithme *est censé* prédire. La manière dont ceci fonctionne est assez intuitive :

Imaginez-vous en train d'essayer d'entrainer votre chien à faire une roulade par exemple. Vous lui montrez comment il doit s'y prendre pour faire la roulade et ensuite il essaye lui même. Suivant sa performance vous lui donnez un nombre de friandises/câlin. Lorsqu'il se rend compte qu'en faisant correctement la roulade, il reçoit beaucoup de friandise, il essayera de recommencer correctement.

Dans cet analogie, le chien représente l'algorithme, la roulade représente la prédiction et les friandises/câlins les modifications apportées aux modèles. Vous montrez à l'algorithme ce qu'il doit atteindre en lui montrant la valeur réelle de ce qu'il doit prédire, il essaye, si il y arrive il ne se corrige que très peu et si il n'y arrive pas, il se corrige beaucoup.

Pour arriver à accomplir cette tâche, il nous faut créer ce training set et le "labeller" c'est à dire calculer les valeurs que le modèle est censé rendre.

Dans le cas de ce notebook, nous avons déjà créé ce dataset (nous avons un dataframe avec toutes les valeurs des prix et des indicateurs déjà stocké) il ne nous reste alors plus qu'à choisir quels indicateurs insérer dans l'algorithme de régression et les agencer de manière matricielle pour que les calculs soit plus aisés (à l'aide de numpy). La matrice sera de ce type, si l'on choisi seulement le prix comme variable :

$$\begin{pmatrix} P_1 & P_2 & P_3 & ... & P_{n} \\ P_2 & P_3 & ... & ... & P_{n+ 1} \\ && \vdots& \\ P_n & ... & ... & ... & P_{2n}\end{pmatrix} * \begin{pmatrix} a_1 \\ a_2 \\ ... \\ a_n \end{pmatrix} = \begin{pmatrix} P_{2n} \\ P_{2n + 1} \\ ... \\ P_{3n} \end{pmatrix} $$

Dans ce cas de figure, nous possédons 2n valeurs et essayons de prédire la valeurs n périodes de temps plus tard.

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
  for X, y in data_iter:
    with tf.GradientTape() as tape:
        l = loss(neural_net(X, training=True), y)
    grads = tape.gradient(l, neural_net.trainable_variables)
    trainer.apply_gradients(zip(grads, neural_net.trainable_variables))
  l = loss(neural_net(features), labels)
  print(f'epoch {epoch + 1}, loss {l:f}')

# 5. Tests :

Il est maintenant le temps de tester les résultats que produisent cette régression. Il est à noter que dans le cas de ce notebook, nous n'avons mis en place qu'un simple algorithme de régression linéaire sur les prix de clôtures, les résultats ne seronts donc probablement pas concluant. Mais il vous est totalement possible d'utiliser l'algorithme sur les indicateurs définis précédemment, il vaus suffit simplement de changer la colonne du dataframe utilisée.

In [ ]:
# Test on another Ticker
values = dic['EURONEXT/PROX']["Last"]
plt.plot(values)
plt.show()

## 5.1 First analysis :

Premièrement, on fait une prédiction avec l'algorithme et on plot cela sur un diagramme (Valeurs réelles, Prédictions) avec une droite x = y sur le diagramme.
Au plus les points sur le diagramme sont proche de la droite et au plus les prédictions seront correctes. Si les points se trouve au dessus de la droite, cela signifie que l'algorithme a sous évalué les prix futurs et si au contraires ils se trouvent en dessus l'algorithme à surévalué les prix.

On peut observer, comme il était prévu, que ces prédictions sont loin d'être précises. Pour augmenter ces précisions, plusieurs possibilités s'ouvrent à nous.

1. Entrainer un réseau de neurones par "configuration de marché" par là j'entend utiliser un réseaux de neurones différents pour les marchés haussiers, baissiers, à hautes volatilités, etc.
2. Entrainer un réseau de neurones par Secteur.
3. Entrainer le réseau de neurones sur différents indicateurs (ce qui sera possible dans le code source, mais ce notebook se veut explicatif, voire introductif par moment).
4. Varier les périodes de temps utilisées.
5. Effectuer des combinaisons des solutions plus haut.

In [ ]:
make_predictions = 30

features = np.array([[values[i+j] for i in range(len(values)-days_forecasting - memory_length - make_predictions, len(values)-days_forecasting - make_predictions)] 
                     for j in range(days_forecasting + make_predictions)])
labels = values[-days_forecasting-make_predictions:]


predictions = neural_net(features)
plt.scatter(predictions, labels)

# Plotting 
plt.plot(labels, labels)
plt.xlabel("Predictions")
plt.ylabel("Values")
plt.show()
print(loss(predictions, labels))
predictions
predictions = tf.reshape(predictions, 32).numpy()

## 5.2 Trend analysis :

Nous avons vu plus haut qu'avec les paramètres utilisés, les résultats étaient imprécis en termes de valeurs, maintenant nous allons voir si l'algorithme permet tout de même d'identifier des tendances.

Le graphe suivant montre les différences de valeurs entre deux jours consécutifs. Lorsque la différence réelle est du même signe que la différence prédite, l'algorithme à prédit correctement la tendence.

On peut observer ici un résultat de 56.25%, ce qui est relativement correct.

In [ ]:
predictions = pd.Series(predictions).diff()
labels = labels.diff()

In [ ]:
def count(predictions, labels):
  counter = 0
  for i in range(len(labels)):
    if (predictions[i] > 0 and labels[i] > 0) or (predictions[i] < 0 and labels[i] < 0):
      counter += 1
  return f'percentage of correctness at predicting tendency : {counter/len(labels)}'
plt.scatter(labels.index, labels, color = "red")
plt.scatter(labels.index, predictions, color = "green")
count(predictions, labels)